In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import folium
from folium.plugins import MiniMap

In [3]:
import json
import altair as alt
import pandas as pd
import geopandas as gpd

In [4]:
ejercicio=pd.read_excel("ejercicio.xlsx" )

In [5]:
#ejercicio

,Unnamed: 0,CVE_MUN,CVE_ENT,ENTIDAD_FEDERATIVA,MUNICIPIO_RESPONSABLE,LAT_DEC,LON_DEC,Clavecompleta,geometry,CLASIFICACION,ESTATUS,Monto por Beneficiario,Desempeño
0,0,1,1,Aguascalientes,Aguascalientes,21.879823,-102.296047,11.0,POLYGON ((2489072.5034999996 1115771.584499999...,Agua y saneamiento,Terminado,193.63,35%
1,1,1,1,Aguascalientes,Aguascalientes,21.879823,-102.296047,11.0,POLYGON ((2489072.5034999996 1115771.584499999...,Agua y saneamiento,Terminado,228.76,20%
2,2,1,1,Aguascalientes,Aguascalientes,21.879823,-102.296047,11.0,POLYGON ((2489072.5034999996 1115771.584499999...,Agua y saneamiento,Terminado,114.40,61%
3,3,1,1,Aguascalientes,Aguascalientes,21.879823,-102.296047,11.0,POLYGON ((2489072.5034999996 1115771.584499999...,Agua y saneamiento,Terminado,76.35,35%
4,4,1,1,Aguascalientes,Aguascalientes,21.879823,-102.296047,11.0,POLYGON ((2489072.5034999996 1115771.584499999...,Agua y saneamiento,Terminado,92.31,61%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100764,100764,58,32,Zacatecas,Santa María de la Paz,21.512178,-103.407042,3258.0,POLYGON ((2356749.0015999973 1062854.583000000...,Transportes y vialidades,Terminado,2997.00,35%
100765,100765,58,32,Zacatecas,Santa María de la Paz,21.512178,-103.407042,3258.0,POLYGON ((2356749.0015999973 1062854.583000000...,Transportes y vialidades,Terminado,3863.43,75%
100766,100766,58,32,Zacatecas,Santa María de la Paz,21.512178,-103.407042,3258.0,POLYGON ((2356749.0015999973 1062854.583000000...,Urbanización,Terminado,16110.40,75%
100767,100767,58,32,Zacatecas,Santa María de la Paz,21.512178,-103.407042,3258.0,POLYGON ((2356749.0015999973 1062854.583000000...,Urbanización,Terminado,59633.06,61%


In [6]:
unique_Edad = ejercicio.ENTIDAD_FEDERATIVA.unique()
unique_Genero = ejercicio.CLASIFICACION.unique()
unique_Estado = ejercicio.ESTATUS.unique()

In [7]:
from ipywidgets import Layout
import ipywidgets as widgets

Estado = widgets.SelectMultiple(
    options = unique_Edad.tolist(),
    #rows=10,
    description='Estado',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)

Clasificacion = widgets.SelectMultiple(
    options = unique_Genero.tolist(),
    #rows=10,
    description='Clasificación',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)

Estatus = widgets.SelectMultiple(
    options = unique_Estado.tolist(),
    #rows=10,
    description='Estatus',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)



In [8]:
def mapaFISM(estado,clasificacion,estatus):
    baseFISM=ejercicio.loc[ejercicio['ENTIDAD_FEDERATIVA']==estado[0]]
    baseFISM=baseFISM.loc[baseFISM['CLASIFICACION']==clasificacion[0]]
    baseFISM=baseFISM.loc[baseFISM['ESTATUS']==estatus[0]]
    #baseFISM=baseFISM.loc[:,['geometry','Monto por Beneficiario','MUNICIPIO_RESPONSABLE','LAT_DEC', 'LON_DEC']]
    baseFISM=baseFISM.drop_duplicates(subset='MUNICIPIO_RESPONSABLE', keep='first', inplace=False)
    baseFISM=baseFISM.reset_index()
    #print(estado)
    #print(clasificacion)
    #print(estatus)
    centro = folium.Map(location=[23.63,-102.55],zoom_start=5)
    for x in range(len(baseFISM)):
        html=f"""
        <h1> {baseFISM['MUNICIPIO_RESPONSABLE'][x]}</h1>
        <p>Variables a considerar:</p>
        <ul>
            <li> Monto por Beneficiario: {baseFISM['Monto por Beneficiario'][x]}</li>
            <li> Desempeño: {baseFISM['Desempeño'][x]}</li>
        </ul>
        </p>
        <p> Link de SIMEPS <a href="http://sistemas.coneval.org.mx/SIMEPS/HomeRamo33.aspx?pCiclo=2022">link</a></p>
        """
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        folium.Marker(
        location=[baseFISM['LAT_DEC'][x],baseFISM['LON_DEC'][x]],
        popup=popup,

        ).add_to(centro)
        #folium.Marker(location=[baseFISM['LAT_DEC'][x],baseFISM['LON_DEC'][x]],
        #popup=[baseFISM['MUNICIPIO_RESPONSABLE'][x],baseFISM['Monto por Beneficiario'][x],baseFISM['Desempeño'][x]]).add_to(centro)
    #print(baseFISM)
    #print(centro)

    display(centro)
    return None
    
    #return print(baseFISM)



In [9]:
import matplotlib.pyplot as plt

In [10]:
widgets.interactive(mapaFISM, estado = Estado, clasificacion=Clasificacion , estatus=Estatus, df= ejercicio)

interactive(children=(SelectMultiple(description='Estado', layout=Layout(height='80px', width='50%'), options=…

In [1]:
#pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
